# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [83]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [84]:
# Take an array of a single feature from the data set and the highest order of degree of polynomial
# Return an sframe of the feature values, with each column power to degree 1, 2, 3, ... to degree n
# For example, with a feature array: 1, 2, 3, 4 and degree 3. We have output: <power_1, power_2, power_3>_1 = <1, 1, 1>
# <power_1, power_2, power_3>_2 = <1, 4, 8>, <power_1, power_2, power_3>_3 = <1, 9, 27>, <power_1, power_2, power_3>_4 = <4, 16, 6>
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature

    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = feature.apply(lambda x: x**power)

    return poly_sframe

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [85]:
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [87]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [88]:
l2_small_penalty = 1e-5
feature_15 = polynomial_sframe(sales['sqft_living'], 15)

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [89]:
feature_15_names = feature_15.column_names()
feature_15['price'] = sales['price']

In [90]:
model15 = graphlab.linear_regression.create(feature_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_small_penalty)

model15

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.119544     | 2662555.739474     | 245656.462167 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 21613
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.1486

Settings
--------
Residual sum of squares        : 1.3042818162e+15
Training RMSE                  : 245656.4622

Highest Positive Coefficients
-----------------------------
(intercept)                    : 167924.8603
power_1                        : 103.0909
power_2                        : 0.1346
power_4                        : 0.0
power_6                        : 0.0

Lowest Negative Coefficients
----------------------------
power_3                       

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?*** power_1: 103.0909

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [91]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [92]:
set_1_15 = polynomial_sframe(set_1['sqft_living'], 15)
set_2_15 = polynomial_sframe(set_2['sqft_living'], 15)
set_3_15 = polynomial_sframe(set_3['sqft_living'], 15)
set_4_15 = polynomial_sframe(set_4['sqft_living'], 15)

In [93]:
set_1_15['price'] = set_1['price']
set_2_15['price'] = set_2['price']
set_3_15['price'] = set_3['price']
set_4_15['price'] = set_4['price']

In [94]:
model_1_15 = graphlab.linear_regression.create(set_1_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_small_penalty)
model_2_15 = graphlab.linear_regression.create(set_2_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_small_penalty)
model_3_15 = graphlab.linear_regression.create(set_3_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_small_penalty)
model_4_15 = graphlab.linear_regression.create(set_4_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_small_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.094320     | 2378292.374908     | 244341.293210 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [95]:
print(model_1_15.get('coefficients')['value'][1])
print(model_2_15.get('coefficients')['value'][1])
print(model_3_15.get('coefficients')['value'][1])
print(model_4_15.get('coefficients')['value'][1])



585.865811236
783.493739085
-759.251897993
1247.59034239


The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.) largest: 1247.59034239 (model_4). smallest: -759.251897993 (model_3)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [96]:
l2_large_penalty=1e5

In [97]:
model_1_15 = graphlab.linear_regression.create(set_1_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_large_penalty)
model_2_15 = graphlab.linear_regression.create(set_2_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_large_penalty)
model_3_15 = graphlab.linear_regression.create(set_3_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_large_penalty)
model_4_15 = graphlab.linear_regression.create(set_4_15, target = 'price', features = feature_15_names, validation_set = None, l2_penalty=l2_large_penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.021379     | 3601895.280124     | 323111.582889 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [98]:
print(model_1_15.get('coefficients')[1:2])
print(model_2_15.get('coefficients')[1:2])
print(model_3_15.get('coefficients')[1:2])
print(model_4_15.get('coefficients')[1:2])

+---------+-------+---------------+---------------+
|   name  | index |     value     |     stderr    |
+---------+-------+---------------+---------------+
| power_1 |  None | 2.58738875673 | 5848.78259004 |
+---------+-------+---------------+---------------+
[1 rows x 4 columns]

+---------+-------+---------------+---------------+
|   name  | index |     value     |     stderr    |
+---------+-------+---------------+---------------+
| power_1 |  None | 2.04470474182 | 10305.6768827 |
+---------+-------+---------------+---------------+
[1 rows x 4 columns]

+---------+-------+---------------+---------------+
|   name  | index |     value     |     stderr    |
+---------+-------+---------------+---------------+
| power_1 |  None | 2.26890421877 | 12779.6057268 |
+---------+-------+---------------+---------------+
[1 rows x 4 columns]

+---------+-------+---------------+---------------+
|   name  | index |     value     |     stderr    |
+---------+-------+---------------+---------------

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.) smallest: 1.91 (model 4) largest: 2.58 (model_1)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [99]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [100]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [101]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2780400035,2014-05-05 00:00:00+00:00,665000.0,4.0,2.5,2800.0,5900,1,0
1703050500,2015-03-21 00:00:00+00:00,645000.0,3.0,2.5,2490.0,5978,2,0
5700002325,2014-06-05 00:00:00+00:00,640000.0,3.0,1.75,2340.0,4206,1,0
0475000510,2014-11-18 00:00:00+00:00,594000.0,3.0,1.0,1320.0,5000,1,0
0844001052,2015-01-28 00:00:00+00:00,365000.0,4.0,2.5,1904.0,8200,2,0
2658000373,2015-01-22 00:00:00+00:00,305000.0,4.0,2.0,1610.0,6250,1,0
3750603471,2015-03-27 00:00:00+00:00,239950.0,3.0,2.5,1560.0,4800,2,0
2114700540,2014-10-21 00:00:00+00:00,366000.0,3.0,2.5,1320.0,4320,1,0
2596400050,2014-07-30 00:00:00+00:00,375000.0,3.0,1.0,1960.0,7955,1,0
4140900050,2015-01-26 00:00:00+00:00,440000.0,4.0,1.75,2180.0,10200,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.

Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [102]:
sf = graphlab.SFrame({'begin':[],'end':[]})

In [103]:
a = list()
a.append(3)
a.append(4)

n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

In [104]:
def create_cross_validation_index(n, k):
    begin_array = list()
    end_array = list()
    
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        begin_array.append(start)
        end_array.append(end)
    
    return begin_array, end_array

validation4 = train_valid_shuffled[5818:7757]

n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

begin_array, end_array = create_cross_validation_index(n, k)

print(begin_array, end_array)

([0, 1939, 3879, 5818, 7758, 9698, 11637, 13577, 15516, 17456], [1938, 3878, 5817, 7757, 9697, 11636, 13576, 15515, 17455, 19395])


To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [105]:
print int(round(validation4['price'].mean(), 0))

536353


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [106]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 |  665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 |  645000.0 |   3.0    |    2.5    |
| 4139480190 | 2014-09-16 00:00:00+00:00 | 1153000.0 |   3.0    |    3.25   |
| 7237300290 | 2015-03-26 00:00:00+00:00 |  338000.0 |   5.0    |    2.5    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   59

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [107]:
train4 = train_valid_shuffled[0:5818].append(train_valid_shuffled[7757+1:n])

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [108]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [127]:
#(train_valid, test) = sales.random_split(.9, seed=1)
#train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

squared = lambda x: x*x

def get_residual_sum_of_squares(model, data, outcome):
    return sum(map(squared, outcome - model.predict(data)))

def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    shuffled_data = graphlab.toolkits.cross_validation.shuffle(data, random_seed=1)
    n = len(shuffled_data)
    rss_average = 0
    
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation = shuffled_data[start:end]
        train = shuffled_data[0:start].append(shuffled_data[end+1:n])
        model = graphlab.linear_regression.create(train, target = output_name, features = features_list, validation_set = None, l2_penalty=l2_penalty)
        rss_average = rss_average + get_residual_sum_of_squares(model, validation, validation[output_name])
        print (rss_average)
    
    return rss_average/k


Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [130]:
import numpy as np
penalties = np.logspace(1, 7, num=13)

feature_15 = polynomial_sframe(sales['sqft_living'], 15)
feature_names = feature_15.column_names()
feature_15['price'] = sales['price']
(train_valid_15, test_15) = feature_15.random_split(.9, seed=1)
lowest_validation_error = float("inf")
best_lambda = 0

for penalty in penalties:
    rss_average = k_fold_cross_validation(k=10, l2_penalty=penalty, data=train_valid_15, output_name = 'price', features_list = feature_names)

    if (rss_average < lowest_validation_error):
        lowest_validation_error = rss_average
        best_lambda = penalty
    
print(best_lambda)


Linear regression:

--------------------------------------------------------

Number of examples          : 17457

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.089579     | 6492747.587172     | 366041.873855 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

2.64676753338e+15
1000.0


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?*** 1000

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [134]:
model_best = graphlab.linear_regression.create(train_valid_15, target = 'price', features = feature_names, validation_set = None, l2_penalty=best_lambda)

rss = get_residual_sum_of_squares(model_best, test, test['price'])

print (rss)

Linear regression:

--------------------------------------------------------

Number of examples          : 19396

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.059131     | 2461778.986191     | 248914.007014 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

2.52897427447e+14
